In [60]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [61]:
# Store filepath in a variable
file = "../Resources/city_data.csv"
# Read our Data file with the pandas library
df = pd.read_csv(file)
df

,ID,City,Lattitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,2112802,Hasaki,35.7333,140.8333,31.69,35.60,55,20,6.91,JP,1612475398
1,779622,Havøysund,70.9963,24.6622,30.20,30.20,86,75,16.11,NO,1612475399
2,1282256,Hithadhoo,-0.6000,73.0833,82.29,82.29,75,85,10.49,MV,1612475243
3,4034551,Faanui,-16.4833,-151.7500,81.32,81.32,68,41,6.17,PF,1612475401
4,2137773,Vao,-22.6667,167.4833,78.73,78.73,86,46,19.82,NC,1612475403
...,...,...,...,...,...,...,...,...,...,...,...
541,1650434,Bambanglipuro,-7.9500,110.2833,76.96,76.96,85,94,3.20,ID,1612480672
542,966380,Pampierstad,-27.7832,24.6877,65.79,65.79,85,0,8.46,ZA,1612480673
543,158514,Kaliua,-5.0606,31.7936,65.30,65.30,94,19,0.45,TZ,1612480674
544,2109701,Auki,-8.7676,160.7034,82.67,82.67,78,79,8.34,SB,1612480675


In [62]:
#D rop null values
df = df.dropna()

# Create a list containing coordinates
coordinates = df[["Lattitude", "Longitude"]].astype(float)

# Convert Humidity to float and store
humidity = df["Humidity (%)"].astype(float)

In [63]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [64]:
#use loc to narrow down data frame to ideal weather criteria
ideal_df = df.loc[(df['Temperature (F)'] >=70 ) & (df["Temperature (F)"]<=80), :]
ideal_df = ideal_df.loc[(ideal_df["Wind Speed (mph)"]<10), :]
ideal_df = ideal_df.loc[(ideal_df["Cloudiness (%)"]==0), :]
ideal_df = ideal_df.dropna()

ideal_df

,ID,City,Lattitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
101,3347019,Mossamedes,-15.1961,12.1522,72.66,72.66,71,0,9.24,AO,1612475263
119,3531368,Celestún,20.8667,-90.4000,76.48,76.48,48,0,4.45,MX,1612475545
202,3395981,Maceió,-9.6658,-35.7353,78.80,78.80,83,0,9.22,BR,1612475648
207,3932145,Pisco,-13.7000,-76.2167,76.57,77.00,73,0,6.91,PE,1612475654
221,105298,Jizan,17.3333,42.6667,77.00,77.00,73,0,9.22,SA,1612475672
274,2336056,Jimeta,9.2833,12.4667,79.09,79.09,17,0,6.24,NG,1612476599
435,3578351,Petites Anses,15.8501,-61.6442,77.86,78.80,73,0,5.75,GP,1612480520
461,2413753,Brikama,13.2714,-16.6494,71.60,71.60,78,0,3.44,GM,1612480561
469,2334008,Kontagora,10.3999,5.4695,78.17,78.17,17,0,7.58,NG,1612480574
513,2361373,Diapaga,12.0708,1.7889,77.43,77.43,18,0,7.47,BF,1612480636


In [65]:
target_coordinates = "-15.1961,12.1522"
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# convert response to json
response = requests.get(base_url, params=params)
places_data = response.json()

print(places_data["results"][0]["geometry"]["location"]["lat"])
print(places_data["results"][0]["geometry"]["location"]["lng"])
print(places_data["results"][0]["name"])

-15.1978317
12.1575544
Hotel Chik Chik Namibe


In [66]:
lat = ideal_df["Lattitude"].tolist()
lng = ideal_df["Longitude"].tolist()
hotels = []
hotel_lat = []
hotel_lng = []

for x in range(len(ideal_df)):
    try:
# geocoordinates
        target_coord=f"{lat[x]}, {lng[x]}"
        target_search = "Hotel"
        target_radius = 5000
        target_type = "lodging"

        params = {
            "location": target_coord,
            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
        }

        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        # convert response to json
        response = requests.get(base_url, params=params)
        places_data = response.json()

        # append lists
        hotels.append(places_data["results"][0]["name"])
        hotel_lat.append(places_data["results"][0]["geometry"]["location"]["lat"])
        hotel_lng.append(places_data["results"][0]["geometry"]["location"]["lng"])

    except: 
        continue

In [67]:
hotels

['Hotel Chik Chik Namibe',
 'Playa 55 beach escape',
 'Jatiúca Hotel & Resort',
 'Apart Hotel Sol de Paracas',
 'Meridien Hotel',
 'Hôtel Bois Joli',
 'Hard Resource lodge']

In [68]:
# Make a data frame with the responses
#Make a dataframe with all the lists
hotel_df = pd.DataFrame({"Hotel" : hotels,
                        "Lattitude": hotel_lat,
                        "Longitude": hotel_lng
                        })
hotel_df


,Hotel,Lattitude,Longitude
0,Hotel Chik Chik Namibe,-15.197832,12.157554
1,Playa 55 beach escape,20.867646,-90.396565
2,Jatiúca Hotel & Resort,-9.641322,-35.699256
3,Apart Hotel Sol de Paracas,-13.717352,-76.209877
4,Meridien Hotel,9.261317,12.454667
5,Hôtel Bois Joli,15.859796,-61.600170
6,Hard Resource lodge,10.406091,5.483965


In [69]:
# Compare coordinates to drop rows of locations that did not get any search results
update_df = ideal_df.drop(221)
update_df = update_df.drop(461)
update_df = update_df.drop(513)
update_df

,ID,City,Lattitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
101,3347019,Mossamedes,-15.1961,12.1522,72.66,72.66,71,0,9.24,AO,1612475263
119,3531368,Celestún,20.8667,-90.4000,76.48,76.48,48,0,4.45,MX,1612475545
202,3395981,Maceió,-9.6658,-35.7353,78.80,78.80,83,0,9.22,BR,1612475648
207,3932145,Pisco,-13.7000,-76.2167,76.57,77.00,73,0,6.91,PE,1612475654
274,2336056,Jimeta,9.2833,12.4667,79.09,79.09,17,0,6.24,NG,1612476599
435,3578351,Petites Anses,15.8501,-61.6442,77.86,78.80,73,0,5.75,GP,1612480520
469,2334008,Kontagora,10.3999,5.4695,78.17,78.17,17,0,7.58,NG,1612480574


In [70]:
# Add hotel names to new dataframes
update_df['Hotels'] = hotels
update_df["Hotel_Lat"] = hotel_lat
update_df["Hotel_Lng"] = hotel_lng
update_df

,ID,City,Lattitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotels,Hotel_Lat,Hotel_Lng
101,3347019,Mossamedes,-15.1961,12.1522,72.66,72.66,71,0,9.24,AO,1612475263,Hotel Chik Chik Namibe,-15.197832,12.157554
119,3531368,Celestún,20.8667,-90.4000,76.48,76.48,48,0,4.45,MX,1612475545,Playa 55 beach escape,20.867646,-90.396565
202,3395981,Maceió,-9.6658,-35.7353,78.80,78.80,83,0,9.22,BR,1612475648,Jatiúca Hotel & Resort,-9.641322,-35.699256
207,3932145,Pisco,-13.7000,-76.2167,76.57,77.00,73,0,6.91,PE,1612475654,Apart Hotel Sol de Paracas,-13.717352,-76.209877
274,2336056,Jimeta,9.2833,12.4667,79.09,79.09,17,0,6.24,NG,1612476599,Meridien Hotel,9.261317,12.454667
435,3578351,Petites Anses,15.8501,-61.6442,77.86,78.80,73,0,5.75,GP,1612480520,Hôtel Bois Joli,15.859796,-61.600170
469,2334008,Kontagora,10.3999,5.4695,78.17,78.17,17,0,7.58,NG,1612480574,Hard Resource lodge,10.406091,5.483965


In [71]:
hotel_info = []
for index, rows in update_df.iterrows():
    hotel_info.append(
        {
            "Hotel": rows.Hotels,
            "City":rows.City,
            "Country":rows.Country,
        }
    )
hotel_info

[{'Hotel': 'Hotel Chik Chik Namibe', 'City': 'Mossamedes', 'Country': 'AO'},
 {'Hotel': 'Playa 55 beach escape', 'City': 'Celestún', 'Country': 'MX'},
 {'Hotel': 'Jatiúca Hotel & Resort', 'City': 'Maceió', 'Country': 'BR'},
 {'Hotel': 'Apart Hotel Sol de Paracas', 'City': 'Pisco', 'Country': 'PE'},
 {'Hotel': 'Meridien Hotel', 'City': 'Jimeta', 'Country': 'NG'},
 {'Hotel': 'Hôtel Bois Joli', 'City': 'Petites Anses', 'Country': 'GP'},
 {'Hotel': 'Hard Resource lodge', 'City': 'Kontagora', 'Country': 'NG'}]

In [72]:
#Set map markers
marker_locations = hotel_df[['Lattitude', 'Longitude']]


# Create map with markers and humidity Heatmap layer
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"{info}" for info in hotel_info])
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(markers)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))